In [1]:
import os
import sys

# change to upper dir to use all custom libs (won't be needed if run from main scripts)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
from scripts.helpers.model_manager import NLLB200Model

model = NLLB200Model(device='cuda')

model.translate_batch(['Счастливые лисички прыгают прыг скок', 'раз два три!'], 'ru', 'kaz')

['Счастливые лисички прыгают прыгают скак Бақытты лисички секіріп секіріп секіреді',
 'Бір, екі, үш рет!']

In [3]:
from scripts.helpers.dataset_manager import HFDataset
import pandas as pd

tt_ru = HFDataset('tat-ru', 'tatar-parallel-162k').get_df()

In [4]:
model._train(
    data=tt_ru,
    src_lang='tat',
    dst_lang='rus',
    training_steps=100,
    save_every=20,
)

  0%|          | 0/100 [00:00<?, ?it/s]

0 1.71875
20 1.701318359375
40 1.668408203125
60 1.578076171875
80 1.41748046875


In [5]:
100*16/

1600

In [9]:
from time import time, sleep

In [7]:
time()

1739913795.951499

In [10]:
start_time = time()
sleep(5)
time() - start_time

5.010152339935303